Text pre processing  techniques such as Replacing new line characters, and segregating text and summary was performed. As per the dataset summary was prefixed with @highlight. So regex was used to split summary and detailed text separately for every article. These are then loaded into Dataframe for further cleaning. Pre processing steps such as lower case conversion, removal of special characters and expansion of contractions was performed. These steps are applied on entire dataframe containing text and summary segregate as 2 rows.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 01 01:11:02 2016
@author: DIP
"""

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
import re
import os
import nltk 
import pandas as pd
import numpy as np
import string
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
  
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^.a-zA-z0-9\s]' if remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern,'', text)
    return text



def normalize_corpus(corpus, contraction_expansion = True, text_lower_case = True, special_char_removal = True,remove_digits=True):

    normalized_corpus = []
    for doc in corpus:
        if text_lower_case:
            doc = doc.lower()

        if special_char_removal:
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits = remove_digits)
        doc = re.sub(' +',' ', doc)
        if contraction_expansion:
            doc = expand_contractions(doc)

        normalized_corpus.append(doc)
    return normalized_corpus

directory = r'/content/sample_data/new'

#text_lst = []# summary_lst = []
t_lst = []
s_lst = []
data_paths = [i for i in (os.path.join(directory, f) for f in os.listdir(directory)) if os.path.isfile(i)]
for file in data_paths: #t_lst = []# s_lst = []
    
    data=open(file,'r')
    
    x = [line.strip('\n').replace("\n"," ") for line in re.split(r'@highlight', data.read() ,re.M)]
    
    summary = x[1: ]
    text = x[0]
   
    summary_str = '.'
    t_lst.append(text)
    s_lst.append(summary_str.join(summary))
    
    

news_df = pd.DataFrame(list(zip(s_lst, t_lst)), columns = ['Summary', 'Text'])


news_df2 = pd.DataFrame(list(zip(normalize_corpus(news_df['Summary']),normalize_corpus(news_df['Text']))),columns = ['clean_sum','clean_text'])
print(news_df2['clean_text'].iloc[0])




despite the huge pressure upon women in the spotlight to appear youthful until well into middle age it seems jennifer aniston is fervently against plastic surgery and cosmetic procedures after seeing the effects on the faces of her celebrity friends . the former friends actress 45 who is rumoured to be tying the knot with fianc justin theroux soon has spoken out against the expectation that women in hollywood be ageless and says she has learned from the mistakes of others when it comes to surgery . speaking to yahoo beauty jen said i think what i have been witness to is seeing women trying to stay ageless with what they are doing to themselves . i am grateful to learn from their mistakes because i am not injecting st into my face . scroll down for video it seems jennifer is against cosmetic procedures after seeing the effects on the faces of her celebrity friends the 45yearold actress said that when she sees other women who have had surgeries her heart breaks honestly i see them and my

For generating abstractive text summarization, Transformer model is used. Its part of Deep learning methodolgy. Using transfer learning, t5model within Transformers is imported and fine tuned for training, test data.
To implement, oad dataset is created using Pytorch framework, importing Dataset class and overriding len and getitem methods. Within getitem, the dataset is accessed and t5tokenizer is applied to convert to tokenids along with attention mask. This is performed on both source text and target summary.

In [ ]:
!pip install pytorch-lightning --quiet
!pip install nlp
!pip install --upgrade pyarrow
!pip install transformers
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from nlp import load_metric

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (3.0.0)


Wandb module is used to assist with debugging and capturing modelpoints for logging. API key used here is generated based on user account

In [ ]:
!pip install wandb
import wandb
YOUR_API_KEY = 'b9ccbb5790e3b4aaac2227be50d35544df4dae18'
os.environ["WANDB_API_KEY"] = YOUR_API_KEY


Load dataset is created using Pytorch framework, importing Dataset class and overriding len and getitem methods. Within getitem, the dataset is accessed and t5tokenizer is applied to convert to tokenids along with attention mask. This is performed on both source text and target summary.


In [ ]:
class Load_dataset(Dataset):
    def __init__(self, dataframe, tokenizer, num_samples, input_length, output_length, print_text=False):         
        self.dataset = dataframe
        #self.dataset =  news_df2
        if num_samples:
            self.dataset = self.dataset.iloc[:num_samples]
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
        self.print_text = print_text
  
    def __len__(self):
        return self.dataset.shape[0]
    
    
    def clean_text(self, text):
        text = text.replace('\n','')
        text = text.replace('``', '')
        text = text.replace('"', '')
        
        return text
    
    
    def convert_to_features(self, example_batch):
        # Tokenize contexts and questions (as pairs of inputs)
        
        if self.print_text:
            print("Input Text: ", self.clean_text(example_batch['clean_text']))
#         input_ = self.clean_text(example_batch['text']) + " </s>"
#         target_ = self.clean_text(example_batch['headline']) + " </s>"
        
        input_ = self.clean_text(example_batch['clean_text'])
        if self.print_text:
            print("Input: ", input_)
            print("Input length: ", self.input_length)
        target_ = self.clean_text(example_batch['clean_sum'])
        
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.output_length, padding='max_length', truncation=True, return_tensors="pt")
    
       
        return source, targets
  
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.dataset.iloc[index])
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()
        #target_ids[target_ids[:,:] == self.tokenizer.pad_token_id] =-100

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()
        

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

Install sentencepice and T5 tokenizer and Conditionforgeneration models from transfromer huggingface library. Here t5-base is used for better performance

In [ ]:
!pip install sentencepiece
!pip install tf-sentencepiece
tokenizer = T5Tokenizer.from_pretrained('t5-base')
#dataset = Load_dataset(tokenizer, None, 512, 150, True)

Train and validate methods are defied which accept epoch, tokenizer along with model, loader and optimizer.
During our model, T5forConditionalgenerator is used and trained by passing input_ids of source text, source attention mask along with decoder input ids and labels. Adam Optimizer is performed with loss is calculated. During test step, model.generate is used to output generated_ids by using tuned param of beam search as 2, repetition_penalty=2.5, length_penalty=1.0. Then generated ids are decoded into predicted summary.

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

Main method invokes training configuraion with batch size, epoch size and learnin rate. Train and test mmodel methods are invoked by splitting datafrae into train/test sizes and invoking dataloader dataset to pass relevenat datasets accodingly.

In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_tutorials_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 4    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 3e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 
    config.adam_epsilon=1e-8

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    #df = pd.read_csv('./data/news_summary.csv',encoding='latin-1')
    #df = df[['text','ctext']]
    df= news_df2
    df.clean_text = 'summarize: ' + df.clean_text
    print(news_df2.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = Load_dataset(train_dataset, tokenizer,None, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = Load_dataset(val_dataset, tokenizer, None, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE, eps=config.adam_epsilon)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    #C:\Users\sa182c\Desktop\personal\python\projects\dailymail\saved_model
    #import os
    torch.save(model, "/content/sample_data/t5")
    #saved_model_dir = f"/content/{MODEL}"

    #saved_model_path = os.path.join(saved_model_dir, max(os.listdir(saved_model_dir)))
    

if __name__ == '__main__':
    main()

wandb: Currently logged in as: swathikakula (use `wandb login --relogin` to force relogin)


                                           clean_sum                                         clean_text
0  the 45yearold has hit out against pressure to ...  summarize: despite the huge pressure upon wome...
1  scarlets hooker ken owens has won 26 caps with...  summarize: wales have called scarlets hooker k...
2  joanne stronach worked in cafe at yorkshire de...  summarize: a motherofthree was sacked from her...
3  information has been redacted from a report in...  summarize: by daniel mills for daily mail aust...
4  bella rodrigueztorres was diagnosed with stage...  summarize: by helen pow published 0916 est 29 ...
FULL Dataset: (152, 2)
TRAIN Dataset: (122, 2)
TEST Dataset: (30, 2)
Initiating Fine-Tuning for the model on our dataset
Epoch: 0, Loss:  6.280526638031006
Epoch: 1, Loss:  1.8290776014328003


AttributeError: ignored